In [13]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [14]:

dataset_path = "Dataset_Brief.csv"
df = pd.read_csv(dataset_path)

In [15]:
missing_rate = df.isna().sum()/df.shape[0]
missing_rate

age         0.0
sex         0.0
bmi         0.0
children    0.0
smoker      0.0
region      0.0
charges     0.0
dtype: float64

In [16]:
df['age'] = df['age'].astype(int)
df['bmi'] = df['bmi'].astype(float)
df['charges'] = df['charges'].astype(float).round(2)
df['sex'].replace(['male', 'female'], [0,1], inplace=True)
df['smoker'].replace(['no', 'yes'], [0,1], inplace=True)
df['is_northwest'] = df['region'].str.contains('northwest').astype(int)
df['is_northeast'] = df['region'].str.contains('northeast').astype(int)
df['is_southwest'] = df['region'].str.contains('southwest').astype(int)
df['is_southeast'] = df['region'].str.contains('southeast').astype(int)
df = df.drop('region', axis=1)
df = df.drop_duplicates(subset=["age", "sex", "bmi", "children", "smoker", 'is_northwest','is_northeast','is_southwest','is_southeast'], keep='first')

df = df[['age','sex','bmi','children','smoker','is_northwest','is_northeast','is_southwest','is_southeast','charges']]
df.head()

,age,sex,bmi,children,smoker,is_northwest,is_northeast,is_southwest,is_southeast,charges
0,19,1,27.900,0,1,0,0,1,0,16884.92
1,18,0,33.770,1,0,0,0,0,1,1725.55
2,28,0,33.000,3,0,0,0,0,1,4449.46
3,33,0,22.705,0,0,1,0,0,0,21984.47
4,32,0,28.880,0,0,1,0,0,0,3866.86


In [17]:
key_columns = ["age",'charges']